In [127]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'gender_submission.csv']


In [128]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [129]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [130]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [131]:
passenger_id=df_test['PassengerId']
df_train.set_index(['PassengerId'], inplace=True)
df_test.set_index(['PassengerId'], inplace=True)

In [132]:
df_train.isnull().sum()


Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [133]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [134]:
df_train.Embarked.fillna('S',inplace=True)

In [135]:
df_test.Fare.fillna(df_test.Fare.mean(),inplace=True)

In [136]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='median', axis=1)
Age2 = imp.fit_transform(df_train['Age'].values.reshape(1,-1))
Age2 = Age2.T
df_train['Age']=Age2
df_train.head()

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [137]:
Age_test = imp.fit_transform(df_test['Age'].values.reshape(1,-1))
Age_test= Age_test.T
df_test['Age']=Age_test
df_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [138]:
df_train.drop(['Cabin'],axis=1, inplace=True)
df_test.drop(['Cabin'],axis=1, inplace=True)

In [139]:
df_train.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [140]:
df_test.isnull().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [141]:
df_train['Sex']=df_train.Sex.apply(lambda x:0 if x=='female' else 1)
df_test['Sex']=df_test.Sex.apply(lambda x:0 if x=='female' else 1)

In [142]:
df_train['Embarked']=df_train.Sex.apply(lambda x:0 if x=='S' else(1 if x=='C' else 2))
df_test['Embarked']=df_test.Sex.apply(lambda x:0 if x=='S' else(1 if x=='C' else 2))

In [143]:
df_train['family_size']=df_train['Parch']+df_train['SibSp']+1
df_test['family_size']=df_test['Parch']+df_test['SibSp']+1

In [144]:
def family_grp(size):
    
    if(size<=1):
        a=0
    elif(size<=4):
        a=1
    else:
        a=2
    return a

df_train['family_size']=df_train.family_size.map(family_grp)
df_test['family_size']=df_test.family_size.map(family_grp)

In [145]:
def age_grp(age):
    
    if(age<=11):
        a=0
    elif(age<=18):
        a=1
    elif(age<=22):
        a=2
    elif(age<=27):
        a=3
    elif(age<=33):
        a=4
    elif(age<=40):
        a=5
    elif(age<=66):
        a=6
    else:
        a=7
    return a
df_train['Age']=df_train['Age'].map(age_grp)
df_test['Age']=df_test['Age'].map(age_grp)

In [146]:
def fare_grp(fare):
    
    if(fare<=7.91):
        a=0
    elif(fare<=14.454):
        a=1
    elif(fare<=31):
        a=2
    elif(fare<=99):
        a=3
    elif(fare<=250):
        a=4
    else:
        a=5
    return a
df_train['Fare']=df_train['Fare'].map(fare_grp)
df_test['Fare']=df_test['Fare'].map(fare_grp)

In [147]:
df_train.drop(['Name','Ticket'],axis=1,inplace=True)
df_test.drop(['Name','Ticket'],axis=1,inplace=True)

In [148]:
df_train['Age_Class']= df_train['Age']* df_train['Pclass']
df_test['Age_Class']= df_test['Age']* df_test['Pclass']

In [149]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,Age_Class
PassengerId,,,,,,,,,,
1,0,3,1,2,1,0,0,2,1,6
2,1,1,0,5,1,0,3,2,1,5
3,1,3,0,3,0,0,1,2,0,9
4,1,1,0,5,1,0,3,2,1,5
5,0,3,1,5,0,0,1,2,0,15


In [150]:
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,Age_Class
PassengerId,,,,,,,,,
892,3,1,5,0,0,0,2,0,15
893,3,0,6,1,0,0,2,1,18
894,2,1,6,0,0,1,2,0,12
895,3,1,3,0,0,1,2,0,9
896,3,0,2,1,1,1,2,1,6


In [151]:
X=df_train.drop('Survived',1)
Y=df_train['Survived']

In [152]:
from sklearn.ensemble import RandomForestClassifier

In [153]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X, Y)

Y_pred = random_forest.predict(df_test)

random_forest.score(X, Y)
acc_random_forest = round(random_forest.score(X, Y) * 100, 2)
print(acc_random_forest)

89.0


In [154]:
submission = pd.DataFrame({
        "PassengerId": passenger_id,
        "Survived": Y_pred
    })

In [155]:
submission.set_index(['PassengerId'], inplace=True)

In [156]:
#import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)